In [1]:
#basic package
import json
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import sys
import shutil
import time
import datetime as dt
import itertools
import time
import re
from collections import Counter
from collections import defaultdict
import operator
import math
from scipy import stats
import pickle
from collections import Counter

import networkx as nx

#feature selections
from sklearn.ensemble.forest import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from scipy.stats import spearmanr, pearsonr

#modelling
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf

#plot
import matplotlib
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf, acf, pacf

from matplotlib.collections import LineCollection
from matplotlib.colors import BoundaryNorm
import seaborn as sns

c:\users\camil\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
from UTILS import perc_element_dico, corr_from_feature2feature, corr_from_dep2feature
import config_mobility as config

we restrict the time series to one value per 60 seconds 
we compute the complexity variables each 30 minutes 
each variables includes the values of at least the last 120.00 minutes (i.e. are using 120.00 values)


In [3]:
path_extracted_data = config.path_extracted_data
path_initial_data = config.path_initial_data
id_run = config.id_run
title_ = '_LB_newword_def_onesession_morethan4_in11_sess_1_2_3' #'_LB', '_all','_LSL', '_LB_newword_def', '_all_newword_def'
path_save_ = os.path.join(path_extracted_data, 'visual', 'predict_KBF', title_)
path_save_LDA = os.path.join(path_extracted_data, 'visual', 'LDA', title_)
path_save_SNA = os.path.join(path_extracted_data, 'visual', 'SNA', 'correlation_graph','LDA',title_)
if not os.path.exists(path_save_SNA):
    os.makedirs(path_save_SNA)

# 1 per bird - KBF prediction

In [4]:
#dataset = read.csv("data-marketing-budget-12mo.csv", header=T, colClasses = c("numeric", "numeric", "numeric"))
df_modelling = pd.read_csv(os.path.join(path_save_,'df_modelling.csv'), sep=";")
print(df_modelling.shape)
df_modelling.head(3)

(57, 540)


,HenID,li_severity,nbr_nan,max_severity,positive_rate_in_severity,max_positive_rate_in_severity,value_of_first_positive_rate_in_severity,li_severity_nonan,var_severity,ratio_HealingAndNonhealing,...,"('Strength', 'mean')","('Mass', 'tuple')","('Mass', 'nbr')","('Mass', 'variance')","('Mass', 'mean')","('Width', 'tuple')","('Width', 'nbr')","('Width', 'variance')","('Width', 'mean')",hybrid
0,hen_100,"[0.0, 0.0, 0.0, 4.5, 4.3, 3.0, 8.5, 8.6, 8.6, ...",0,8.7,"[4.5, 5.5, 0.09999999999999964, 0.099999999999...",5.5,4.5,"[0.0, 0.0, 0.0, 4.5, 4.3, 3.0, 8.5, 8.6, 8.6, ...",13.076529,0.166667,...,42.750000,"(59.6, 58.6, 67.7, 66.9)",4.0,17.015000,63.200000,"(0.3, 0.36, 0.3, 0.32)",3.0,0.000600,0.320000,LB
1,hen_101,"[3.0, 2.9, 2.6, 5.1, 6.1, 6.2, 9.8, 9.4, 9.8, ...",0,9.8,"[2.4999999999999996, 1.0, 0.10000000000000053,...",3.6,2.5,"[3.0, 2.9, 2.6, 5.1, 6.1, 6.2, 9.8, 9.4, 9.8, ...",8.417025,0.118421,...,48.600000,"(57.4, 58.2, 58.3, 56.0, 55.6)",5.0,1.240000,57.100000,"(0.31, 0.3, 0.26, 0.31, 0.27)",4.0,0.000440,0.290000,LB
2,hen_102,"[0.0, 0.0, 0.0, 2.0, 4.3, 3.2, 2.8, 2.9, 3.2, ...",0,4.6,"[2.0, 2.3, 0.10000000000000009, 0.300000000000...",2.3,2.0,"[0.0, 0.0, 0.0, 2.0, 4.3, 3.2, 2.8, 2.9, 3.2, ...",2.660165,0.393443,...,49.666667,"(64.0, 63.6, 60.4)",3.0,2.595556,62.666667,"(0.34, 0.33, 0.36)",3.0,0.000156,0.343333,LB


## Correlation

In [5]:
li_var = [x for x in df_modelling.columns if ('sess_1' in x) & \
          (('day0' in x) | ('day1' in x) | ('day2' in x)) & \
          ('topic_proba' not in x)]
li_var = [x for x in df_modelling.columns if 'sess_' in x]
li_cat = [x for x in li_var if x.endswith('_topic')]
li_cont = [x for x in li_var if (x not in li_cat) & ('proba' in x)]
print(len(li_var), len(li_cont), len(li_cat))
li_cont[0:2], li_cat[0:2]

714 378 54


(['sess_1_day0_k2_topic_proba', 'sess_1_day0_k2_topic_0_proba'],
 ['sess_1_day0_k2_topic', 'sess_1_day0_k3_topic'])

In [6]:
li_init = ['HenID', 'li_severity', 'nbr_nan','li_severity_nonan','positive_rate_in_severity',
           "('Strength', 'tuple')", "('Strength', 'nbr')", "('Mass', 'tuple')", "('Mass', 'nbr')", 
           "('Width', 'tuple')", "('Width', 'nbr')", 'hybrid','ratio_HealingAndNonhealing_round']
print(len(li_init))
li_output = [x for x in df_modelling.columns if (x not in li_init) & (x not in li_var)]
print(len(li_output))
#li_output = ['max_severity']
li_output

13
11


['max_severity',
 'max_positive_rate_in_severity',
 'value_of_first_positive_rate_in_severity',
 'var_severity',
 'ratio_HealingAndNonhealing',
 "('Strength', 'variance')",
 "('Strength', 'mean')",
 "('Mass', 'variance')",
 "('Mass', 'mean')",
 "('Width', 'variance')",
 "('Width', 'mean')"]

In [7]:
df_significance = corr_from_dep2feature(li_output, li_cont, df_modelling, p_val=0.05)
df_significance.to_csv(os.path.join(path_save_LDA, 'correlation_cont_cont.csv'),sep=';')
print(df_significance.shape)
df_significance.head(3)

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:11<00:00,  1.06s/it]


-------------------------------------------------------------------
There is 0 potential issues, and 282 significant correlation
(282, 6)


,var1,var2,val_spear,pval_spear,val_pers,pval_pers
0,max_severity,sess_1_day0_k6_topic_3_proba,-0.089281,0.516847,-0.285150,0.034843
1,max_severity,sess_1_day0_k7_topic_4_proba,-0.275005,0.042152,-0.333231,0.012917
2,max_severity,sess_1_day0_k8_topic_2_proba,-0.191341,0.161694,-0.295892,0.028281


In [8]:
li_df = []
for x,df_ in tqdm.tqdm(df_significance.groupby(['var1'])):
    li = df_['var2'].tolist()
    df_feature_feature = corr_from_feature2feature(li, df_modelling, p_val=0.05)
    li_df.append(df_feature_feature)
df_feature_feature = pd.concat(li_df)
df_feature_feature.to_csv(os.path.join(path_save_LDA, 'correlation_feature_feature.csv'),sep=';')
print(df_feature_feature.shape)
df_feature_feature.head(3)

  0%|                                                                                           | 0/11 [00:00<?, ?it/s]

-------------------------------------------------------------------
There is 0 potential issues, and 213 significant correlation


  9%|███████▌                                                                           | 1/11 [00:00<00:09,  1.06it/s]

-------------------------------------------------------------------
There is 0 potential issues, and 155 significant correlation


 18%|███████████████                                                                    | 2/11 [00:01<00:08,  1.08it/s]

...

 91%|██████████████████████████████████████████████████████████████████████████▌       | 10/11 [00:12<00:01,  1.18s/it]

-------------------------------------------------------------------
There is 0 potential issues, and 158 significant correlation


100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:13<00:00,  1.12s/it]


(262, 6)


,var1,var2,val_spear,pval_spear,val_pers,pval_pers
0,sess_1_day0_k8_topic_4_proba,sess_1_day0_k9_topic_5_proba,0.345751,9.722721e-03,0.412650,1.743317e-03
1,sess_1_day0_k9_topic_5_proba,sess_1_day0_k9_topic_8_proba,-0.633562,2.084403e-07,-0.609814,7.732939e-07
2,sess_1_day0_k9_topic_8_proba,sess_1_day4_k5_topic_proba,0.246655,7.217384e-02,0.365586,6.559134e-03


# Correlation Graph

In [9]:
df_feature_feature = pd.read_csv(os.path.join(path_save_LDA, 'correlation_feature_feature.csv'), sep=';', index_col=0)
print(df_feature_feature.shape)
display(df_feature_feature.head(3))

df_significance = pd.read_csv(os.path.join(path_save_LDA, 'correlation_cont_cont.csv'), sep=';', index_col=0)
print(df_significance.shape)
display(df_significance.head(3))

df_corr = pd.concat([df_significance, df_feature_feature])
print(df_corr.shape)
df_corr.head(3)

(262, 6)


,var1,var2,val_spear,pval_spear,val_pers,pval_pers
0,sess_1_day0_k8_topic_4_proba,sess_1_day0_k9_topic_5_proba,0.345751,9.722721e-03,0.412650,1.743317e-03
1,sess_1_day0_k9_topic_5_proba,sess_1_day0_k9_topic_8_proba,-0.633562,2.084403e-07,-0.609814,7.732939e-07
2,sess_1_day0_k9_topic_8_proba,sess_1_day4_k5_topic_proba,0.246655,7.217384e-02,0.365586,6.559134e-03


(282, 6)


,var1,var2,val_spear,pval_spear,val_pers,pval_pers
0,max_severity,sess_1_day0_k6_topic_3_proba,-0.089281,0.516847,-0.285150,0.034843
1,max_severity,sess_1_day0_k7_topic_4_proba,-0.275005,0.042152,-0.333231,0.012917
2,max_severity,sess_1_day0_k8_topic_2_proba,-0.191341,0.161694,-0.295892,0.028281


(544, 6)


,var1,var2,val_spear,pval_spear,val_pers,pval_pers
0,max_severity,sess_1_day0_k6_topic_3_proba,-0.089281,0.516847,-0.285150,0.034843
1,max_severity,sess_1_day0_k7_topic_4_proba,-0.275005,0.042152,-0.333231,0.012917
2,max_severity,sess_1_day0_k8_topic_2_proba,-0.191341,0.161694,-0.295892,0.028281


In [10]:
dico_nodename_attribute = {x:{'type_':'feature'} for x in set(list(df_corr['var1'])+list(df_corr['var2']))}
for x in li_output:
    dico_nodename_attribute[x] = {'type_':'dependant'}
for x in dico_nodename_attribute.keys():
    if 'day' in x:
        dico_nodename_attribute[x]['day'] = int(x.split('day')[1].split('_')[0])
        dico_nodename_attribute[x]['sess'] = int(x.split('_')[1])
        dico_nodename_attribute[x]['k'] = int(x.split('_k')[1].split('_')[0])
print(len(dico_nodename_attribute))

210


In [11]:
#compute a graph of correlations to make it easier
def correlationGraph(df_corr, dico_nodename_attribute, path_save_SNA, name_='', p_val_spear=0.01, p_val_pers=0.01, 
                     condition_type='or'):
    '''From a dataframe of correlation, compute the associated graph, where
    input: dataframe with the following columns: var1, var2, val_spear, pval_spear, val_pers, pval_pers
    link: if two nodes have a significant p-values (p_val in parameters will be used)
    node: each var1, var2*
    edge attribute: weight_coeff: correlation coefficient, weight_pval: correlation p-value
    node attribute: type: feature/dependant
    condition_type: can only be "and", or "or", it will raise an error otherwise '''

    #small test
    if condition_type not in ['and','or']:
        print('ERROR: condition_type: can only be "and", or "or"')
        sys.exit()
        
    #initialise graph
    G = nx.Graph() #only one edge between two nodes (otherwise use MultiGraph)

    #add nodes with its attribute
    dico_name_id = {}
    dico_n_a = {}
    for i, (n, dico_attribute) in enumerate(dico_nodename_attribute.items()):
        dico_name_id[n] = i
        dico_n_a[i] = dico_attribute.copy()
        dico_n_a[i]['name'] = n
        G.add_node(i) 
    nx.set_node_attributes(G, dico_n_a)
    print(len(dico_n_a))
    print(dico_n_a[3])
  
    #add edges
    li_edges = [] 
    for i in range(df_corr.shape[0]):
        x = df_corr.iloc[i]
        n1 = x['var1'] ; n2 = x['var2'] ; pval_spear = x['pval_spear'] ; pval_pers = x['pval_pers']
        #if significant, link it
        if ((pval_spear<=p_val_spear) & (pval_pers<=p_val_pers) & (condition_type=='and')) |\
           (((pval_spear<=p_val_spear) | (pval_pers<=p_val_pers)) & (condition_type=='or')):
            li_edges.append((dico_name_id[n1], dico_name_id[n2], {'pval_spear':float(round(pval_spear,3)),
                                      'pval_pers':float(round(pval_pers,3)), 
                                      'val_spear':float(round(x['val_spear'],3)), 
                                      'val_pers':float(round(x['val_pers'],3))}))
    #add edges with attributes
    G.add_edges_from(li_edges)
    print(len(li_edges))
    #write G (networkX graph) in GEXF format for gephi
    nx.write_gexf(G, os.path.join(path_save_SNA, name_+'_'+condition_type+'_'+'.gexf'))    

In [12]:
name_ = ''
if len(li_output)==1:
    name_ = li_output[0]
correlationGraph(df_corr, dico_nodename_attribute, path_save_SNA=path_save_SNA, 
                 name_=name_, p_val_spear=0.01, p_val_pers=0.01, 
                 condition_type='or')

210
{'type_': 'feature', 'day': 2, 'sess': 1, 'k': 10, 'name': 'sess_1_day2_k10_topic_7_proba'}
275


In [13]:
TODO

NameError: name 'TODO' is not defined

### Modelling

In [ ]:
import sklearn
from sklearn import linear_model

#linear regression
df = df.dropna(subset=['f_13','total_kb_emails','ping_pong_perc','chi2_distance_max','team_location_diversity',
        'Leistungsbeurteilung 2015_number','synchronical_duration_perc','num_sent_t','betweenness_full'])


X = df[['f_13']]
y = df[['total_kb_emails','ping_pong_perc','chi2_distance_max','team_location_diversity',
        'Leistungsbeurteilung 2015_number','synchronical_duration_perc','num_sent_t','betweenness_full']]

# Create linear regression object
regr = linear_model.LinearRegression(normalize=True)

# Train the model
regr.fit(X, y)

# The coefficients
print('Coefficients: \n', regr.coef_)

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(X, y))


In [ ]:
df_modelling.isna().sum()
#duration_zone_1!!!!

In [ ]:
df_modelling.head(3)

In [ ]:
x = 'max_severity' 

li_mlp = ['duration_zone_1', 'duration_zone_2','duration_zone_3', 'duration_zone_4', 'duration_zone_5',
           'FirstTimestamp_zone_1_h', 'FirstTimestamp_zone_2_h',
           'FirstTimestamp_zone_3_h', 'FirstTimestamp_zone_4_h',
           'FirstTimestamp_zone_5_h', 'Total_number_zone','Max_duration_zones',
           'Max_duration', 'Min_duration', 'Median_duration', 'Average_duration',
           'Variance_duration', 'dico_zone_duration', 'Total_number_transition',
           'nbr_bouts_zone_5', 'nbr_bouts_zone_4',
           'nbr_bouts_zone_3', 'nbr_bouts_zone_2', 'nbr_bouts_zone_1',
           'Max_duration_zone_4']
li_cpx = ['SampEnt_order2', 'SampEnt_zone_1','SampEnt_zone_2', 'SampEnt_zone_3', 'SampEnt_zone_4', 'SampEnt_zone_5',
          'RunSampEnt_onLastTsOfEachLevel', 'RunDistEnt_onLastTsOfEachLevel',
          'RunSampEnt_onLastTsOfEachLevel_1', 'RunSampEnt_onLastTsOfEachLevel_2',
          'RunSampEnt_onLastTsOfEachLevel_3', 'RunSampEnt_onLastTsOfEachLevel_4',
          'RunSampEnt_onLastTsOfEachLevel_5',
          'distribution_entropy']    

li_choosen_mlp = ['duration_zone_1', 'FirstTimestamp_zone_4_h', 'Median_duration', 'Variance_duration','SampEnt_order2']

li_cluster_lda_day1 = [x for x in df_modelling.columns if ('sess_1_day0' in x) & ('k6' in x) & ('proba' in x)\
                       & ('topic_proba' not in x)]
li_mlp_day1 = [x for x in df_modelling.columns if ('sess_1_day0' in x) & ('topic' not in x) \
               & any(i in x for i in li_choosen_mlp)]
li_var = list(set(['is_LSL'] + li_cluster_lda_day1 + li_mlp_day1))
li_var

In [ ]:
df_modelling['is_LSL'] = df_modelling['hybrid'].map(lambda x: x=='LSL')
g = sns.PairGrid(df_modelling[[x]+li_var], hue='is_LSL')
g.map(plt.scatter);
plt.savefig(os.path.join(path_save_,id_run+'_scatterplot_var.png'), format='png', dpi=300)
#Total_number_transition, duration_zone_2 , SampEnt_zone_4up --> perc_after24 up and perc_before24 down
#Max_duration_zone_4, duration_zone_4 up --> erc_after24 down and perc_before24 up
#??Variance_duration?? fucked up  --'SampEnt_order2','SampEnt_zone_4' down

In [ ]:
df_modelling['is_LSL'] = df_modelling['hybrid'].map(lambda x: x=='LSL')
for day in range(0,6):
    print(day)
    for t in range(min_topic,max_topic+1):
        df_ = df_modelling[['max_severity','is_LSL']+['sess_1_day'+str(day)+'_k'+str(t)+'_topic_'+str(n)+'_proba' for n in range(t)]]
        df_ = pd.melt(df_,id_vars=['max_severity','is_LSL']) 
        sns.lmplot(x="max_severity", y='value', hue='is_LSL', col='variable', data=df_, 
                   ci=95); #categorical x: x_estimator=np.mean, x_bins=5, x_ci=95, x_estimator=np.mean)
        #order: order=2 or lowess=True
        #logistic=True: if y is binary
        #truncate=True
        #robust=True
        #x_partial=is_LSL
        plt.savefig(os.path.join(path_save_,id_run+'sess_1_day'+str(day)+'k_'+str(t)+'_topic_proba.png'), format='png', dpi=300)
        plt.show()

In [ ]:
li_var = ['sess_1_day0_k6_topic_5_proba',
 'sess_1_day0_Variance_duration',
 'sess_1_day0_Median_duration',
 'sess_1_day0_duration_zone_1',
 'sess_1_day0_k6_topic_0_proba']
li_var = ['sess_1_day0_k6_topic_5_proba', 'sess_1_day0_Variance_duration', 'sess_1_day0_duration_zone_1']

In [ ]:
#GLM
formula = x+' ~ '+'+'.join(li_var)
min_sample_Size = 50 + 8*len(li_var)
df_mod = df_modelling[(~df_modelling[x].isnull())&(df_modelling['is_LSL']==False)][li_var+[x,'HenID']].copy()
display(df_mod.head(3))
print(min_sample_Size, df_mod.shape)
if min_sample_Size>=df_mod.shape[0]:
    print('WARNING: reduce the number of dependant variables or augment your number of observation')
#    sys.exit()
    
mod = smf.glm(formula=formula, data=df_modelling).fit() #, family=sm.families.Binomial()
mod.summary()
#--> mass seems to increase with higher percentgae of shift 22

sst_val = sum(map(lambda x: np.power(x,2),y-np.mean(y))) 
sse_val = sum(map(lambda x: np.power(x,2),m1.resid_response)) 
r2 = 1.0 - sse_val/sst_val

In [ ]:
### RandomForest
x = 'max_severity' 

li_mlp = ['duration_zone_1', 'duration_zone_2','duration_zone_3', 'duration_zone_4', 'duration_zone_5',
           'FirstTimestamp_zone_1_h', 'FirstTimestamp_zone_2_h',
           'FirstTimestamp_zone_3_h', 'FirstTimestamp_zone_4_h',
           'FirstTimestamp_zone_5_h', 'Total_number_zone','Max_duration_zones',
           'Max_duration', 'Min_duration', 'Median_duration', 'Average_duration',
           'Variance_duration', 'dico_zone_duration', 'Total_number_transition',
           'nbr_bouts_zone_5', 'nbr_bouts_zone_4',
           'nbr_bouts_zone_3', 'nbr_bouts_zone_2', 'nbr_bouts_zone_1',
           'Max_duration_zone_4']
li_cpx = ['SampEnt_order2', 'SampEnt_zone_1','SampEnt_zone_2', 'SampEnt_zone_3', 'SampEnt_zone_4', 'SampEnt_zone_5',
          'RunSampEnt_onLastTsOfEachLevel', 'RunDistEnt_onLastTsOfEachLevel',
          'RunSampEnt_onLastTsOfEachLevel_1', 'RunSampEnt_onLastTsOfEachLevel_2',
          'RunSampEnt_onLastTsOfEachLevel_3', 'RunSampEnt_onLastTsOfEachLevel_4',
          'RunSampEnt_onLastTsOfEachLevel_5',
          'distribution_entropy']    

li_choosen_mlp = ['duration_zone_1', 'FirstTimestamp_zone_4_h', 'Median_duration', 'Variance_duration','SampEnt_order2']

li_cluster_lda_day1 = [x for x in df_modelling.columns if ('sess_1_day0' in x) & ('k2' in x) & ('proba' in x)\
                       & ('topic_proba' not in x)]
li_mlp_day1 = [x for x in df_modelling.columns if ('sess_1_day0' in x) & ('topic' not in x) \
               & any(i in x for i in li_choosen_mlp)]
li_var = list(set(li_cluster_lda_day1 + li_mlp_day1)) #['is_LSL'] + 
li_var = li_cluster_lda_day1
li_var

df_ = df_modelling[(~df_modelling[x].isnull()) & (~df_modelling['sess_1_day0_k4_topic_3_proba'].isnull())][li_var+[x]]
print(df_.shape, df_modelling.shape)
df_['sess_1_day0_k2_topic_1_proba'] = df_['sess_1_day0_k2_topic_1_proba'].map(lambda x: round(x,2))
df_['sess_1_day0_k2_topic_0_proba'] = df_['sess_1_day0_k2_topic_0_proba'].map(lambda x: round(x,2))
df_['max_severity'] = df_['max_severity'].map(lambda x: round(x/10,1))

display(df_)
print(df_.isna().sum())
X_train, X_test, y_train, y_test = train_test_split(df_modelling[li_var], df_modelling[x], test_size=0.2, random_state=40)
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)
sel = SelectFromModel(RandomForestClassifier(n_estimators=20))
sel.fit(X_train, y_train)
sel.get_support()
selected_feat= X_train.columns[(sel.get_support())]
len(selected_feat)

df_test.fillna(df_test.mean())
X_test = df_test.values  
y_pred = clf.predict(X_test)